In [160]:
import arcpy

In [185]:
from arcgis.gis import GIS

In [186]:
import json

In [196]:
from arcgis.features import FeatureLayerCollection

In [201]:
from arcgis.features import FeatureLayer

In [10]:
import os

In [114]:
arcpy.env.overwriteOutput = True

In [2]:
camp_site = 'https://services5.arcgis.com/9s1YtFmLS0YTl10F/ArcGIS/rest/services/ZHomeless_Camp_Trash_Collector/FeatureServer/0'

In [213]:
flayer = gis.content.search('ZHomeless_Camp_Trash_Collector')[2]

In [220]:
outpath = r'G:\projects\UtilityDistricts\eweb\DrinkingWater\IllegalCampCoordination\Recieved'

In [3]:
path = outpath + '\\IllegalCampNotification_pro'

In [13]:
sdeFile = path + "\\RLIDDB.sde"

In [43]:
conn = arcpy.CreateDatabaseConnection_management(out_folder_path=path,
                                          out_name="RLIDDB.sde",
                                          database_platform="SQL_SERVER",
                                          instance="rliddb.int.lcog.org,5433",
                                          account_authentication="OPERATING_SYSTEM_AUTH",
                                          database="RLIDGeo")

In [41]:
if os.path.exists(sdeFile):
    os.remove(sdeFile)

In [44]:
arcpy.env.workspace = conn.getOutput(0)

In [205]:
fc = "RLIDGeo.DBO.Taxlot"

In [152]:
listdates1 = []

In [156]:
cursor = arcpy.da.SearchCursor(camp_site, "Date")

In [157]:
for row in cursor:
    listdates1.append(row)

In [62]:
from datetime import datetime

In [158]:
type(listdates1[0][0])

datetime.datetime

In [215]:
listdates1[1:4]

[(datetime.datetime(2016, 5, 18, 15, 38, 19, 867000),),
 (datetime.datetime(2016, 5, 18, 15, 39, 57, 341000),),
 (datetime.datetime(2016, 5, 16, 15, 42, 32, 964000),)]

In [68]:
listdates = [dt[0] for dt in listdates1 if dt[0] is not None]

In [69]:
maxdate = max(listdates)

In [70]:
maxdate

datetime.datetime(2023, 1, 18, 22, 52, 35, 515000)

In [71]:
maxdatestr = str(maxdate)

In [216]:
maxdatestr

'2023-01-18 22:52:35.515000'

In [219]:
datestr = maxdatestr.split('.')[0].split(' ')[0]

In [81]:
timestrlist = maxdatestr.split(' ')[1].split(':')

In [82]:
h = timestrlist[0]
m = timestrlist[1]
s = timestrlist[2].split('.')[0]

In [83]:
def convert_hour(str_h):
    h = int(str_h)
    if h >= 0 and h <= 11:
        res = [str_h, 'AM']
    elif h == 12:
        res = [str_h, 'PM']
    elif h > 12 and h <=23:
        res = [str(h-12), 'PM']
    return res

In [86]:
cvt_h = convert_hour(h)

In [222]:
def convert_date(datestr):
    datestrlist = datestr.split('-')
    Y = datestrlist[0]
    m = str(int(datestrlist[1]))
    d = str(int(datestrlist[2]))
    res = m+'/'+d+'/'+Y
    return res, Y, m, d

In [226]:
res = convert_date(datestr)

In [227]:
Y = res[1]

In [228]:
m = res[2]

In [229]:
d = res[3]

In [231]:
d

'18'

In [232]:
outfolder = os.path.join(outpath, Y, m+'_'+d)

In [235]:
if os.path.exists(outfolder):
    print('No need to report!')

No need to report!


In [223]:
convert_date(datestr)[0]

'1/18/2023'

In [173]:
def getDateStr(date):
    def convert_hour(str_h):
        h = int(str_h)
        if h >= 0 and h <= 11:
            res = [str_h, 'AM']
        elif h == 12:
            res = [str_h, 'PM']
        elif h > 12 and h <=23:
            res = [str(h-12), 'PM']
        return res
    
    if date is None:
        res = None
    else:
        strdate = str(date)
        datestr = strdate.split(' ')[0]
        res = convert_date(datestr)
    return res

In [96]:
maxdatestr_n = convert_date(datestr) + ' '  + cvt_h[0] + ':' + m + ':' + s + ' ' + cvt_h[1]

In [97]:
maxdatestr_n

'1/18/2023 10:52:35 PM'

In [137]:
getDateStr(maxdate)

'1/18/2023 10:52:35 PM'

In [100]:
f"Date = '{maxdatestr_n}'"

"Date = '1/18/2023 10:52:35 PM'"

In [129]:
arcpy.management.CopyFeatures(camp_site, path + '\\MyProject4.gdb\\datacopy')

<Result 'G:\\projects\\UtilityDistricts\\eweb\\DrinkingWater\\IllegalCampCoordination\\Recieved\\IllegalCampNotification_pro\\MyProject4.gdb\\datacopy'>

In [134]:
arcpy.management.AddFields(path + '\\MyProject4.gdb\\datacopy', [['Date_str', "TEXT", 'datestr', 255, '1/18/2023 10:52:35 PM', '']])

<Result 'G:\\projects\\UtilityDistricts\\eweb\\DrinkingWater\\IllegalCampCoordination\\Recieved\\IllegalCampNotification_pro\\MyProject4.gdb\\datacopy'>

In [171]:
expression = "getDateStr(!Date!)"

codeblock = """
def getDateStr(date):
    def convert_date(datestr):
        datestrlist = datestr.split('-')
        Y = datestrlist[0]
        m = str(int(datestrlist[1]))
        d = str(int(datestrlist[2]))
        res = m+'/'+d+'/'+Y
        return res

    if date is None:
        res = None
    else:
        strdate = str(date)
        datestr = strdate.split(' ')[0]
        res = convert_date(datestr)
    return res"""

In [172]:
arcpy.management.CalculateField(path + '\\MyProject4.gdb\\datacopy', 'Date_str',  expression, "PYTHON3", 
                                codeblock)

<Result 'G:\\projects\\UtilityDistricts\\eweb\\DrinkingWater\\IllegalCampCoordination\\Recieved\\IllegalCampNotification_pro\\MyProject4.gdb\\datacopy'>

In [165]:
maxdatestr_n

'1/18/2023 10:52:35 PM'

In [169]:
maxdate

datetime.datetime(2023, 1, 18, 22, 52, 35, 515000)

In [174]:
getDateStr(maxdate)

'1/18/2023'

In [175]:
selres = arcpy.SelectLayerByAttribute_management(path + '\\MyProject4.gdb\\datacopy',
                                                 "NEW_SELECTION", f"Date_str = '{getDateStr(maxdate)}'")

In [176]:
print(selres)

datacopy_Layer5


In [177]:
arcpy.management.CopyFeatures(selres, path + '\\MyProject4.gdb\\most_recent')

<Result 'G:\\projects\\UtilityDistricts\\eweb\\DrinkingWater\\IllegalCampCoordination\\Recieved\\IllegalCampNotification_pro\\MyProject4.gdb\\most_recent'>

In [240]:
arcpy.analysis.SpatialJoin(selres, fc, path + '\\MyProject4.gdb\\HomelessCampSite_SpatialJoin14c', "JOIN_ONE_TO_ONE", "KEEP_ALL",'Status "Status" true true false 50 Text 0 0,First,#,Homeless Camp Site,Status,0,50;Comments "Comments" true true false 500 Text 0 0,First,#,Homeless Camp Site,Comments,0,500;Date "Date" true true false 8 Date 0 0,First,#,Homeless Camp Site,Date,-1,-1;Submitted_by "Submitted By" true true false 50 Text 0 0,First,#,Homeless Camp Site,Submitted_by,0,50;Dogs_present "Dogs present" true true false 50 Text 0 0,First,#,Homeless Camp Site,Dogs_present,0,50;Unruly_inhabitants "Unruly inhabitants" true true false 50 Text 0 0,First,#,Homeless Camp Site,Unruly_inhabitants,0,50;Hazardous_materials_present "Hazardous materials present" true true false 50 Text 0 0,First,#,Homeless Camp Site,Hazardous_materials_present,0,50;Biohazards_present "Biohazards present" true true false 50 Text 0 0,First,#,Homeless Camp Site,Biohazards_present,0,50;Size_of_encampment "Size of encampment" true true false 50 Text 0 0,First,#,Homeless Camp Site,Size_of_encampment,0,50;GlobalID "GlobalID" false false true 38 GlobalID 0 0,First,#,Homeless Camp Site,GlobalID,-1,-1;maptaxlot_hyphen "maptaxlot_hyphen" true true false 17 Text 0 0,First,#,Lane County Taxlots,maptaxlot_hyphen,0,17;ownname "ownname" true true false 128 Text 0 0,First,#,Lane County Taxlots,ownname,0,128;addr1 "addr1" true true false 64 Text 0 0,First,#,Lane County Taxlots,addr1,0,64;ownercity "ownercity" true true false 40 Text 0 0,First,#,Lane County Taxlots,ownercity,0,40;ownerprvst "ownerprvst" true true false 30 Text 0 0,First,#,Lane County Taxlots,ownerprvst,0,30;ownerzip "ownerzip" true true false 10 Text 0 0,First,#,Lane County Taxlots,ownerzip,0,10;geocity_name "geocity_name" true true false 32 Text 0 0,First,#,Lane County Taxlots,geocity_name,0,32;ugb_name "ugb_name" true true false 32 Text 0 0,First,#,Lane County Taxlots,ugb_name,0,32;longitude "longitude" true true false 8 Double 8 38,First,#,Lane County Taxlots,longitude,-1,-1;latitude "latitude" true true false 8 Double 8 38,First,#,Lane County Taxlots,latitude,-1,-1', "INTERSECT", None, '')

<Result 'G:\\projects\\UtilityDistricts\\eweb\\DrinkingWater\\IllegalCampCoordination\\Recieved\\IllegalCampNotification_pro\\MyProject4.gdb\\HomelessCampSite_SpatialJoin14c'>

In [242]:
arcpy.conversion.TableToExcel(path + '\\MyProject4.gdb\\HomelessCampSite_SpatialJoin14c', path+'\\most_recent.xlsx')

<Result 'G:\\projects\\UtilityDistricts\\eweb\\DrinkingWater\\IllegalCampCoordination\\Recieved\\IllegalCampNotification_pro\\most_recent.xlsx'>